# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [1]:
# Your solution goes hereimport requests
from bs4 import BeautifulSoup
import pandas as pd
import re # For regular expressions, helpful in cleaning price and UPC

def scrape_books(min_rating: int, max_price: float) -> pd.DataFrame:
    """
    Scrapes book data from Books to Scrape, then filters the results
    based on the minimum rating and maximum price criteria.

    Parameters:
    - min_rating (int): Minimum star rating (1 to 5).
    - max_price (float): Maximum price in Pounds (£).

    Returns:
    - pd.DataFrame: DataFrame containing filtered book data.
    """
    base_url = "http://books.toscrape.com/"
    current_url = base_url + "catalogue/page-1.html"
    all_books_data = []

    print(f"Starting scrape from: {base_url}")
    print(f"Filtering criteria: Min Rating >= {min_rating}, Max Price <= £{max_price:.2f}")

    # --- 1. Main Pagination Loop ---
    # The loop continues as long as a 'next' button is found on the page
    while True:
        try:
            # Fetch the page content
            response = requests.get(current_url)
            response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
            soup = BeautifulSoup(response.text, 'html.parser')
        except requests.exceptions.RequestException as e:
            print(f"Error fetching page {current_url}: {e}")
            break

        print(f"-> Processing page: {current_url.split('/')[-1]}...")

        # Find all book articles on the current page
        book_articles = soup.find_all('article', class_='product_pod')

        # --- 2. Extract Book Details Loop (Per Page) ---
        for article in book_articles:
            book_data = {}

            # Relative link to the book's detail page
            relative_link = article.h3.a['href']
            # Build the absolute URL for the detail page
            detail_url = base_url + 'catalogue/' + relative_link.replace('../', '')

            # Scrape Title, Price, and Rating from the main listing page
            book_data['Title'] = article.h3.a['title']
            
            # Price: Extract text and clean it to a float
            price_text = article.find('p', class_='price_color').text
            book_data['Price (£)'] = float(re.search(r'[\d\.]+', price_text).group())

            # Rating: Convert class name to integer rating
            rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
            rating_class = article.find('p', class_=re.compile(r'star-rating'))['class'][-1]
            book_data['Rating'] = rating_map.get(rating_class, 0)
            
            
            # --- 3. Scrape Detail Page for UPC, Genre, Availability, and Description ---
            try:
                detail_response = requests.get(detail_url)
                detail_response.raise_for_status()
                detail_soup = BeautifulSoup(detail_response.text, 'html.parser')
            except requests.exceptions.RequestException as e:
                print(f"Error fetching detail page {detail_url}: {e}")
                continue # Skip to the next book

            # UPC, Availability (Table Data)
            # All these are in the 'Product Information' table
            product_info_table = detail_soup.find('table', class_='table-striped')
            if product_info_table:
                # Find all table rows
                rows = product_info_table.find_all('tr')
                
                # UPC is in the first row
                book_data['UPC'] = rows[0].find('td').text
                
                # Availability is typically in the 6th row (Index 5)
                availability_text = rows[5].find('td').text
                # Clean up availability string (e.g., 'In stock (22 available)')
                book_data['Availability'] = availability_text.strip()
            else:
                book_data['UPC'] = None
                book_data['Availability'] = "N/A"

            # Genre (Breadcrumb navigation)
            # The genre is the third item in the breadcrumb list (index 2)
            breadcrumb_list = detail_soup.find('ul', class_='breadcrumb')
            if breadcrumb_list and len(breadcrumb_list.find_all('li')) >= 3:
                book_data['Genre'] = breadcrumb_list.find_all('li')[2].a.text
            else:
                book_data['Genre'] = "Unknown"

            # Description
            description_tag = detail_soup.find('div', id='product_description')
            if description_tag:
                # The actual description is the next sibling paragraph after the <div>
                book_data['Description'] = description_tag.find_next_sibling('p').text
            else:
                book_data['Description'] = "No description available."
            
            all_books_data.append(book_data)

        # --- 4. Find the next page link ---
        next_link = soup.find('li', class_='next')
        if next_link:
            # Construct the full URL for the next page
            next_page_relative = next_link.a['href']
            current_url = base_url + 'catalogue/' + next_page_relative
        else:
            # No 'next' button found, end the loop
            break
            
    # --- 5. Data Structuring and Filtering ---
    df = pd.DataFrame(all_books_data)
    
    # Filter the DataFrame based on the function parameters
    # Note: The data type of 'Price (£)' is float, 'Rating' is int/float
    filtered_df = df[
        (df['Rating'] >= min_rating) & 
        (df['Price (£)'] <= max_price)
    ].reset_index(drop=True)
    
    print("\nScraping complete.")
    print(f"Total books scraped: {len(df)}")
    print(f"Filtered books meeting criteria: {len(filtered_df)}")
    
    return filtered_df

# --- Execution based on Task Requirements ---

# Run the script with the required parameters: min_rating >= 4 and max_price <= £20
MIN_RATING_THRESHOLD = 4
MAX_PRICE_THRESHOLD = 20.0

# 

try:
    books_df = scrape_books(
        min_rating=MIN_RATING_THRESHOLD,
        max_price=MAX_PRICE_THRESHOLD
    )
    
    # Display the first 5 results of the filtered DataFrame
    print("\n## 📚 Filtered Books Data (First 5 Rows) 📚")
    if not books_df.empty:
        print(books_df.head().to_markdown(index=False))
        print(f"\nDataFrame shape: {books_df.shape}")
    else:
        print("No books matched the specified criteria.")

except Exception as e:
    print(f"\nAn unexpected error occurred during execution: {e}")

Starting scrape from: http://books.toscrape.com/
Filtering criteria: Min Rating >= 4, Max Price <= £20.00

An unexpected error occurred during execution: name 'requests' is not defined


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests # <--- THIS IS REQUIRED
from bs4 import BeautifulSoup # <--- THIS IS REQUIRED
import pandas as pd # <--- THIS IS REQUIRED
import re
# ... rest of the script (function definition and execution)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re # For regular expressions, helpful in cleaning price and UPC

def scrape_books(min_rating: int, max_price: float) -> pd.DataFrame:
    """
    Scrapes book data from Books to Scrape, then filters the results
    based on the minimum rating and maximum price criteria.

    Parameters:
    - min_rating (int): Minimum star rating (1 to 5).
    - max_price (float): Maximum price in Pounds (£).

    Returns:
    - pd.DataFrame: DataFrame containing filtered book data.
    """
    base_url = "http://books.toscrape.com/"
    current_url = base_url + "catalogue/page-1.html"
    all_books_data = []

    print(f"Starting scrape from: {base_url}")
    print(f"Filtering criteria: Min Rating >= {min_rating}, Max Price <= £{max_price:.2f}")

    # --- 1. Main Pagination Loop ---
    # The loop continues as long as a 'next' button is found on the page
    while True:
        try:
            # Fetch the page content
            response = requests.get(current_url)
            response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
            soup = BeautifulSoup(response.text, 'html.parser')
        except requests.exceptions.RequestException as e:
            print(f"Error fetching page {current_url}: {e}")
            break

        print(f"-> Processing page: {current_url.split('/')[-1]}...")

        # Find all book articles on the current page
        book_articles = soup.find_all('article', class_='product_pod')

        # --- 2. Extract Book Details Loop (Per Page) ---
        for article in book_articles:
            book_data = {}

            # Relative link to the book's detail page
            relative_link = article.h3.a['href']
            # Build the absolute URL for the detail page
            detail_url = base_url + 'catalogue/' + relative_link.replace('../', '')

            # Scrape Title, Price, and Rating from the main listing page
            book_data['Title'] = article.h3.a['title']
            
            # Price: Extract text and clean it to a float
            price_text = article.find('p', class_='price_color').text
            book_data['Price (£)'] = float(re.search(r'[\d\.]+', price_text).group())

            # Rating: Convert class name to integer rating
            rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
            rating_class = article.find('p', class_=re.compile(r'star-rating'))['class'][-1]
            book_data['Rating'] = rating_map.get(rating_class, 0)
            
            
            # --- 3. Scrape Detail Page for UPC, Genre, Availability, and Description ---
            try:
                detail_response = requests.get(detail_url)
                detail_response.raise_for_status()
                detail_soup = BeautifulSoup(detail_response.text, 'html.parser')
            except requests.exceptions.RequestException as e:
                print(f"Error fetching detail page {detail_url}: {e}")
                continue # Skip to the next book

            # UPC, Availability (Table Data)
            # All these are in the 'Product Information' table
            product_info_table = detail_soup.find('table', class_='table-striped')
            if product_info_table:
                # Find all table rows
                rows = product_info_table.find_all('tr')
                
                # UPC is in the first row
                book_data['UPC'] = rows[0].find('td').text
                
                # Availability is typically in the 6th row (Index 5)
                availability_text = rows[5].find('td').text
                # Clean up availability string (e.g., 'In stock (22 available)')
                book_data['Availability'] = availability_text.strip()
            else:
                book_data['UPC'] = None
                book_data['Availability'] = "N/A"

            # Genre (Breadcrumb navigation)
            # The genre is the third item in the breadcrumb list (index 2)
            breadcrumb_list = detail_soup.find('ul', class_='breadcrumb')
            if breadcrumb_list and len(breadcrumb_list.find_all('li')) >= 3:
                book_data['Genre'] = breadcrumb_list.find_all('li')[2].a.text
            else:
                book_data['Genre'] = "Unknown"

            # Description
            description_tag = detail_soup.find('div', id='product_description')
            if description_tag:
                # The actual description is the next sibling paragraph after the <div>
                book_data['Description'] = description_tag.find_next_sibling('p').text
            else:
                book_data['Description'] = "No description available."
            
            all_books_data.append(book_data)

        # --- 4. Find the next page link ---
        next_link = soup.find('li', class_='next')
        if next_link:
            # Construct the full URL for the next page
            next_page_relative = next_link.a['href']
            current_url = base_url + 'catalogue/' + next_page_relative
        else:
            # No 'next' button found, end the loop
            break
            
    # --- 5. Data Structuring and Filtering ---
    df = pd.DataFrame(all_books_data)
    
    # Filter the DataFrame based on the function parameters
    # Note: The data type of 'Price (£)' is float, 'Rating' is int/float
    filtered_df = df[
        (df['Rating'] >= min_rating) & 
        (df['Price (£)'] <= max_price)
    ].reset_index(drop=True)
    
    print("\nScraping complete.")
    print(f"Total books scraped: {len(df)}")
    print(f"Filtered books meeting criteria: {len(filtered_df)}")
    
    return filtered_df

# --- Execution based on Task Requirements ---

# Run the script with the required parameters: min_rating >= 4 and max_price <= £20
MIN_RATING_THRESHOLD = 4
MAX_PRICE_THRESHOLD = 20.0

# 

try:
    books_df = scrape_books(
        min_rating=MIN_RATING_THRESHOLD,
        max_price=MAX_PRICE_THRESHOLD
    )
    
    # Display the first 5 results of the filtered DataFrame
    print("\n## 📚 Filtered Books Data (First 5 Rows) 📚")
    if not books_df.empty:
        print(books_df.head().to_markdown(index=False))
        print(f"\nDataFrame shape: {books_df.shape}")
    else:
        print("No books matched the specified criteria.")

except Exception as e:
    print(f"\nAn unexpected error occurred during execution: {e}")

Starting scrape from: http://books.toscrape.com/
Filtering criteria: Min Rating >= 4, Max Price <= £20.00
-> Processing page: page-1.html...
-> Processing page: page-2.html...
-> Processing page: page-3.html...
-> Processing page: page-4.html...
-> Processing page: page-5.html...
-> Processing page: page-6.html...
